## [CLVM values](https://chialisp.com/docs/clvm/basics#clvm-values)

CLVM is built out of `cons boxes` and `atoms`. These are referred to as **CLVM Objects**. A `cons box` is a pair of **CLVM Objects**. The items in a `cons box` can either be an `atom` or another `cons box`.

### [Atoms](https://chialisp.com/docs/clvm/basics#atoms)
An atom is a string of bytes.

### [Cons Boxes](https://chialisp.com/docs/clvm/basics#cons-boxes)
Cons boxes are represented as a parentheses with two elements separated by a `.`. For example:

## [List]()
A list is a representation of consecutive cons boxes terminated in a null atom `()`.

`(200 . (300 . (400 . ())))` -> `(200 300 400)`

`(+ . ((q . 1) . ((q . 2) . ())))` 
-> `(+ . ((q . 1) . ((q . 2))))`
-> `(+ . ((q . 1) (q . 2)))`
-> `(+ (q . 1) (q . 2))` 

In [2]:
%%bash
brun '(q . (200 . (300 . (400 . ()))))'

(200 300 400)


In [14]:
%%bash
brun '(q . (+ . ((q . 1) . ((q . 2) . ()))))'
brun '(q . (+ . ((q . 1) . ((q . 2)))))'
brun '(q . (+ . ((q . 1) (q . 2))))'
brun '(q . (+ (q . 1) (q . 2)))'

(+ (q . 1) (q . 2))
(+ (q . 1) (q . 2))
(+ (q . 1) (q . 2))
(+ (q . 1) (q . 2))


## [S-expression](https://en.wikipedia.org/wiki/S-expression)

In the usual parenthesized syntax of Lisp, an S-expression is classically defined as

1. an `atom`, or
1. an expression of the form `(x . y)` where x and y are S-expressions.

## [SExp](https://github.com/Chia-Network/clvm/blob/main/clvm/SExp.py#L120)
SExp provides higher level API on top of any object implementing the **CLVM object protocol**.
    
The CLVM object protocol (concept) exposes two attributes:

    1. `atom` which is either `None` or `bytes`
    2. `pair` which is either `None` or a `tuple` of exactly two elements. Both elements implementing the CLVM object protocol.
    
Exactly one of `atom` and `pair` must be `None`.

## [Program](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/types/blockchain_format/program.py#L22)

A thin wrapper around `s-expression` data intended to be invoked with "eval".

### [compile_clvm_text](https://github.com/Chia-Network/clvm_tools/blob/main/clvm_tools/clvmc.py#L14)

## [CLVMObject](https://github.com/Chia-Network/clvm/blob/main/clvm/CLVMObject.py#L4)

```python
class CLVMObject:
    """
    This class implements the CLVM Object protocol in the simplest possible way,
    by just having an "atom" and a "pair" field
    """

    atom: typing.Optional[bytes]

    # this is always a 2-tuple of an object implementing the CLVM object
    # protocol.
    pair: typing.Optional[typing.Tuple[typing.Any, typing.Any]]
```

In [78]:
from chia.types.blockchain_format.program import Program
from clvm_tools.binutils import disassemble
from clvm_tools.clvmc import compile_clvm_text
from clvm.operators import KEYWORD_FROM_ATOM

clsp_text = '''
(mod ARGUMENT (+ ARGUMENT 175))
'''

puzzle: Program = Program(
    compile_clvm_text(clsp_text, search_paths=[])
)

print(puzzle)
print(disassemble(puzzle))

# (+ 1 (q . 175))
# (+ . (1 . ((q . 175) . ())))

#    []
#   /  \
#  +   []
#     /  \
#    1   []
#       /  \
#      []  ()
#     /  \
#    q   175  


print(puzzle.atom)
print(puzzle.pair[0].as_python()) # +
print(puzzle.pair[1].pair[0].as_python()) # 1
print(puzzle.pair[1].pair[1].pair[1].atom) # ''
print(puzzle.pair[1].pair[1].pair[0].pair[0].atom) # q
print(puzzle.pair[1].pair[1].pair[0].pair[1].atom) # 175


print(KEYWORD_FROM_ATOM[puzzle.pair[0].as_python()])
print(puzzle.pair[1].pair[0].as_python())
print(KEYWORD_FROM_ATOM[puzzle.pair[1].pair[1].pair[0].pair[0].atom])
print(puzzle.pair[1].pair[1].pair[0].pair[1].atom)


print(disassemble(puzzle.pair[1].pair[1].pair[0]))
print((puzzle.pair[1].pair[1].pair[1]()))

print(int('00af', 16))


ff10ff01ffff018200af80
(+ 1 (q . 175))
None
b'\x10'
b'\x01'
b''
b'\x01'
b'\x00\xaf'
+
b'\x01'
q
b'\x00\xaf'
(q . 175)


AttributeError: 'CLVMObject' object has no attribute 'as_python'

In [71]:
%%bash
brun '(q . (+ . (1 . ((q . 175) . ()))))'

(+ 1 (q . 175))


## [Program Evaluation](https://chialisp.com/docs/clvm/lang_reference/#program-evaluation)

The semantics of the language implemented by the CLVM is similar to Lisp. A program is represented as a binary tree. The root of the tree is the least nested object in the program tree, with inner function calls embedded recursively inside of it. In the following example, the outer parentheses represent the `cons box` that is the root of the tree `(+ (q . 1) (q . 2))`.


```
      [ ]
     /   \
    +     [ ]
         /   \
      [q, 1]  [ ]
             /   \
         [q, 2]  nil
```